In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

In [ ]:
df = pd.read_csv('livecell_base_preprocessing_old.csv')

In [ ]:
df

In [ ]:
df_BT474 = df[df.cell_type=='BT474']

In [ ]:
df_BT474

In [ ]:
df_counts = df.groupby(['image_id']).size()
display(df_counts.head())
ax = df_counts.hist(bins=50, grid=True)
ax.set_xlabel("Annotations per image")
ax.set_ylabel("Images with particular annot. count")

In [ ]:
df = pd.read_csv('livecell_base_preprocessing.csv')

In [ ]:
df

In [ ]:
df_train = df[df['original_split']=='train']
df_val = df[df['original_split']=='val']
df_test = df[df['original_split']=='test']
df_list = [df_train, df_val, df_test]

In [ ]:
df_counts = df_train.groupby(['image_id']).size()
display(df_counts.head())
ax = df_counts.hist(bins=50, grid=True)
ax.set_xlabel("Annotations per image")
ax.set_ylabel("Images with particular annot. count")

In [ ]:
df_counts = df_val.groupby(['image_id']).size()
display(df_counts.head())
ax = df_counts.hist(bins=50, grid=True)
ax.set_xlabel("Annotations per image")
ax.set_ylabel("Images with particular annot. count")

In [ ]:
df_counts = df_test.groupby(['image_id']).size()
display(df_counts.head())
ax = df_counts.hist(bins=50, grid=True)
ax.set_xlabel("Annotations per image")
ax.set_ylabel("Images with particular annot. count")

In [ ]:
df_livecell_cell_types = df.groupby('image_id')['cell_type'].first().reset_index()
visualize_cell_type_distributions(df=df_livecell_cell_types, title='Cell Type Distribution in LIVECell Dataset')

In [ ]:
df_livecell_cell_types = df_train.groupby('image_id')['cell_type'].first().reset_index()
visualize_cell_type_distributions(df=df_livecell_cell_types, title='Cell Type Distribution in LIVECell Dataset')

In [ ]:
df_livecell_cell_types = df_val.groupby('image_id')['cell_type'].first().reset_index()
visualize_cell_type_distributions(df=df_livecell_cell_types, title='Cell Type Distribution in LIVECell Dataset')

In [ ]:
df_livecell_cell_types = df_test.groupby('image_id')['cell_type'].first().reset_index()
visualize_cell_type_distributions(df=df_livecell_cell_types, title='Cell Type Distribution in LIVECell Dataset')

In [ ]:
df_livecell_cell_types = df_test.groupby('cell_type')['image_id']
visualize_cell_type_distributions(df=df_livecell_cell_types, title='Cell Type Distribution in LIVECell Dataset')

In [ ]:
import numpy as np
import math
import os
from matplotlib.path import Path

array = np.zeros((520, 704))
array_help = df[df['image_id']==1564017]["polygons"].to_numpy()
# print(type(df[df['image_id']=='1564017']['polygons']))
# print(type(array_help[0]))
for img_mask in array_help:
#     print(type(img_mask))
#     print(img_mask)
    img_mask = img_mask[2:-2]
    img_mask = np.array(img_mask.split())
#     print(type(img_mask))
#     print(img_mask)
    x = img_mask[0::2]
    y = img_mask[1::2]
    
    arr = [(x, y) for (x, y) in zip(y,x)]
    vertices = np.asarray(arr)
    path = Path(vertices)
    xmin, ymin, xmax, ymax = np.asarray(path.get_extents(), dtype=int).ravel()
    x, y = np.mgrid[:520, :704]
    
    # mesh grid to a list of points
    points = np.vstack((x.ravel(), y.ravel())).T

    # select points included in the path
    mask = path.contains_points(points)
    path_points = points[np.where(mask)]
    print(mask)
    # reshape mask for display
    img_mask = mask.reshape(x.shape)
    img_mask = img_mask.astype(np.int)
    array += img_mask

plt.figure(figsize=(15,15))
plt.imshow(array.clip(0, 1))

In [ ]:
def mask_decode(mask):
    array = np.zeros((520, 704))
    for label in mask:
        s = label.split()
        starts = list(map(lambda x: int(x) - 1, s[0::2]))
        lengths = list(map(int, s[1::2]))
        ends = [x + y for x, y in zip(starts, lengths)]
        img = np.zeros((520*704), dtype=np.float32)            
        for start, end in zip(starts, ends):
            img[start : end] = 1 
        array += img.reshape((520, 704))
    return array.clip(0, 1)

def rle_encode(img):
    """ TBD
    
    Args:
        img (np.array): 
            - 1 indicating mask
            - 0 indicating background
    
    Returns: 
        run length as string formated
    """
    
    pixels = img.flatten()
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)

PatientInfoTuple = namedtuple(
    'PatientInfoTuple',
    'id, cell_type, annotations'
)

def getPatientsInfo():
    df = pd.read_csv('../input/sartorius-cell-instance-segmentation/train.csv')
    patientsInfo_list = list()
    
    for id in df.id.unique():
        cell_type = df[df.id == id].cell_type.unique()[0]
        annotations = df[df["id"] == id]["annotation"].tolist()
        
        patientsInfo_list.append(PatientInfoTuple(
            id,
            cell_type,
            annotations
        ))
        
    return patientsInfo_list

In [ ]:
seg_list = list()
array_help = df[df['image_id']==1564017]["polygons"].to_numpy()
# print(type(df[df['image_id']=='1564017']['polygons']))
print(type(array_help[0]))
for img_mask in array_help:
#     print(type(img_mask))
#     print(img_mask)
    img_mask = img_mask[2:-2]
    img_mask = np.array(img_mask.split())
    print(type(img_mask))

    x = img_mask[0::2]
    y = img_mask[1::2]
    
    arr = [(x, y) for (x, y) in zip(y,x)]
    vertices = np.asarray(arr)
    path = Path(vertices)
    xmin, ymin, xmax, ymax = np.asarray(path.get_extents(), dtype=int).ravel()
    x, y = np.mgrid[:520, :704]
    
    # mesh grid to a list of points
    points = np.vstack((x.ravel(), y.ravel())).T

    # select points included in the path
    mask = path.contains_points(points)
    path_points = points[np.where(mask)]

    # reshape mask for display
    img_mask = mask.reshape(x.shape)
    img_mask = img_mask.astype(np.int)
    # ENCODED MASK
    encoded_img_mask = rle_encode(img_mask)
    seg_list.append(encoded_img_mask)


seg_list[0]